In [ ]:
import triton
import triton.language as tl
@triton.jit 
def montecarloint(y_samples_ptr,result_ptr,a,b,n_samples,BLOCK_SIZE:tl.constexpr):
    y_samples_ptr = y_samples_ptr.to(tl.pointer_type(tl.float32))
    result_ptr = result_ptr.to(tl.pointer_type(tl.float32))
    pid = tl.program_id(0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    mask = offsets < n_samples
    x = tl.load(y_samples_ptr + offsets, mask=mask)
    val=tl.sum(x,axis=0)
    contribution = (b - a) * val / n_samples
    tl.atomic_add(result_ptr, contribution)


def solve(y_samples_ptr: int, result_ptr: int, a: float, b: float, n_samples: int):
    BLOCK_SIZE = 512
    grid = (triton.cdiv(n_samples, BLOCK_SIZE),)
    montecarloint[grid](
        y_samples_ptr, result_ptr, a,b, n_samples,
        BLOCK_SIZE=BLOCK_SIZE
    )
    
